# Cross-comparisons on Cecil
*Tom Walker (tom[at]cecil.earth)*

This notebook accompanies the Cecil newsletter ***[Dataset Evaluation](https://newsletter.cecil.earth/p/operating-in-spite-of-uncertainty)***. 

The notebook walks through doing some basic cross-comparisons of three plant biomass datasets on Cecil. The notebook:

* Creates an AOI
* Makes data requests for three datasets
* Transforms the datasets to the same CRS and spatial resolution
* Queries the data and calculates summary statistics
* Creates three basic cross-comparison visualisations:
    * Pairwise scatter graphs showing correlations of pixel AGB values across datasets
    * Difference maps showing spatial variation in AGB values among datasets
    * Time series graphs showing temporal variation in AGB values among datasets

The notebook assumes you have followed the first three steps of the [getting started](https://docs.cecil.earth/Getting-started-111ef16bbbe48123aaa1d0a4bbd0a63d) guide to:

1. Install the Cecil SDK.
2. Sign up for an account.
3. Configure the SDK with your user API key.


Once this is complete, begin by importing `GeoPandas`, `Matplotlib` and `Cecil`.



In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams['axes.formatter.useoffset'] = False # removes offset on plots

import cecil
client = cecil.Client()

___

## Create an AOI, make data requests and transform data

### Create an AOI

Use the code below to create an AOI on your account. Replace `aoi_name` with your own name and `aoi_geometry` with a geojson object containing a polygon of the AOI boundary.

Store the AOI ID (`aoi.id`) so you can retrieve it later (see [SDK documentation](https://docs.cecil.earth/SDK-11fef16bbbe480b5a778ccae542cf34d) for details).

*NB - you only need to create an AOI once.*

In [ ]:
# store aoi name and geometry (geojson polygon)
aoi_name = 'Kakadu National Park'
aoi_geometry = {
        "type": "Polygon",
        "coordinates": [
            [
                [132.52934211276073, -12.721072673008706],
                [132.52934211276073, -12.730063400794094],
                [132.54027735328083, -12.730063400794094],
                [132.54027735328083, -12.721072673008706],
                [132.52934211276073, -12.721072673008706]
            ]
        ]
}

# create aoi on cecil
aoi = client.create_aoi(
    name=aoi_name,
    geometry=aoi_geometry
)

In [ ]:
# print aoi id
print(f'AOI ID: {aoi.id}')

### Make data requests

For this exercise, we will request data from three datasets:

* [Chloris Aboveground Biomass Stock and Change (30 m)](https://docs.cecil.earth/Aboveground-biomass-stock-and-change-30-m-111ef16bbbe48163a502ed2bcb64fc72)
* [Kanop Screening (25 m)](https://docs.cecil.earth/Screening-25-m-111ef16bbbe481a780bce4de81a500a5)
* [Planet Forest Carbon Diligence dataset (30 m)](https://docs.cecil.earth/Forest-carbon-diligence-111ef16bbbe481d9a430c446c2bb4d04)

The following code block stores dataset IDs and list prices (links above) and uses the AOI size (`aoi.hectares`) to estimate the total data acquisition cost for your AOI.

In [ ]:
# get dataset ids
chloris_dataset_id = 'e837b8b1-1002-4ff1-a4ca-fb6a7ac928fd'
kanop_dataset_id = 'f79af205-e45d-45b2-99fb-5108d83bd6f5'
planet_dataset_id = '53738a57-a889-43c9-8f7a-7cb306831700'

# get acquisition costs
chloris_dataset_price = 0.09
kanop_dataset_price = 0.10
planet_dataset_price = 0.10

# print cost
cost = chloris_dataset_price * aoi.hectares + kanop_dataset_price * aoi.hectares + planet_dataset_price * aoi.hectares
print(f'Approximate acquisition cost: ${round(cost, 2)}')

Use the following code block to make a data request for each dataset (dataset IDs) against your AOI (`aoi.id`). 

Store data request IDs (`data_request.id`) so you can retrieve them later (see [SDK documentation](https://docs.cecil.earth/SDK-11fef16bbbe480b5a778ccae542cf34d)).

*NB - you only need to make the data requests once. It may take some hours for a data request to complete.*

In [ ]:
# request datasets
chloris_request = client.create_data_request(
    aoi_id=aoi.id,
    dataset_id=chloris_dataset_id
)

kanop_request = client.create_data_request(
    aoi_id=aoi.id,
    dataset_id=kanop_dataset_id
)

planet_request = client.create_data_request(
    aoi_id=aoi.id,
    dataset_id=planet_dataset_id
)

In [ ]:
# print data request ids
print(f'Chloris data request ID: {chloris_request.id}')
print(f'Kanop data request ID: {kanop_request.id}')
print(f'Planet data request ID: {planet_request.id}')

### Transform  the data

Datasets often have different native coordinate reference systems (CRS) and spatial resolutions, which must be aligned before working with them together. 

Use the following code to transform all datasets to the same CRS and spatial resolution. Here we have chosen EPSG:4326 and 0.00025º (i.e. ~ 30 m on the equator).

Store transformation IDs (`transformation.id`) so you can retrieve them later (see [SDK documentation](https://docs.cecil.earth/SDK-11fef16bbbe480b5a778ccae542cf34d)).

*NB - you only need to do the transformations once.*

In [ ]:
# choose transformation crs and spatial resolution
destination_crs = 'EPSG:4326'
destination_res = 0.00025

# transform datasets
chloris_transformation = client.create_transformation(
    data_request_id=chloris_request.id,
    crs=destination_crs,
    spatial_resolution=destination_res
)

kanop_transformation = client.create_transformation(
    data_request_id=kanop_request.id,
    crs=destination_crs,
    spatial_resolution=destination_res
)

planet_transformation = client.create_transformation(
    data_request_id=planet_request.id,
    crs=destination_crs,
    spatial_resolution=destination_res
)

In [ ]:
# print data request ids
print(f'Chloris transformation ID: {chloris_transformation.id}')
print(f'Kanop transformation ID: {kanop_transformation.id}')
print(f'Planet transformation ID: {planet_transformation.id}')

---

## Query and wrangle the data

### Query data

Retrieve your transformation IDs and use them to query the data from your Cecil database. 

In [ ]:
# store transformation ids (convenience)
chloris_tid = chloris_transformation.id
kanop_tid = kanop_transformation.id
planet_tid = planet_transformation.id

The SQL query below:

* Filters for selected transformation IDs
* Joins the datasets by pixel centroids (x, y) and year
* Selects aboveground biomass (AGB) variables from all providers, as well as pixel and year information
* Transforms Planet AGB from a carbon equivalent to total biomass (see [dataset documentation](https://docs.cecil.earth/Forest-carbon-diligence-111ef16bbbe481d9a430c446c2bb4d04))
* Returns the data into a data frame in your Python environment

In [ ]:
df = client.query(f'''
    SELECT
        -- Basic variables
        x, y, year,
        ST_ASWKT(pixel_boundary) AS pixel_boundary,
        -- Biomass variables
        k.living_aboveground_biomass AS kanop_agb,
        p.aboveground_live_carbon_density / 0.476 AS planet_agb,
        c.aboveground_biomass_stock AS chloris_agb
    FROM transformation_db.kanop.screening_25_m k
    -- Join all datasets
    JOIN transformation_db.planet.forest_carbon_diligence p USING (year, x, y)
    JOIN transformation_db.chloris.aboveground_biomass_stock_and_change_30_m c USING (year, x, y)
    -- Filter for chosen transformations
    WHERE 
        (
            k.transformation_id = '{kanop_tid}' OR
            p.transformation_id = '{planet_tid}' OR
            c.transformation_id = '{chloris_tid}'
        )
''')

### Wrangle data

Convert the data into a GeoPandas data frame for downstream analysis and print the top 3 rows:

In [ ]:
# convert data frame to geodataframe
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.GeoSeries.from_wkt(
        df['pixel_boundary'], 
        crs='EPSG:4326'
    )
)

# inspect data frame
df.head(3)

The code below creates the mean and coefficient of variation of AGB across providers for use in cross-comparisons at two levels:

* AOI level - all pixels together, separating years
* Pixel level - all years together, separating pixels

In [ ]:
# calculate mean among the three variables
gdf['mean_agb'] = gdf[['kanop_agb', 'planet_agb', 'chloris_agb']].apply(
    lambda row: row.mean(), 
    axis=1
)

# calculate coefficient of variation among the three variables
gdf['cv_agb'] = gdf[['kanop_agb', 'planet_agb', 'chloris_agb']].apply(
    lambda row: row.std() / row.mean(), 
    axis=1
)

# create df with mean per pixel coefficient of variation across all years (pixel level)
spatial_cv = gdf.groupby(['x', 'y']).agg({
    'cv_agb': 'mean',
    'geometry': 'first'
}).reset_index()

# create df with annual mean cv and annual mean/sd of agb (aoi level)
temporal_stats = gdf.groupby('year').agg({
    'cv_agb': 'mean',
    'mean_agb': ['mean', 'std']
}).reset_index()
temporal_stats.columns = ['year', 'mean_cv', 'mean_agb', 'std_mean_agb']

---

## Cross-comparisons

The following code blocks perform three basic visual cross-comparisons:

1. Scatter graphs showing pairwise pixel correlations between all providers
2. Difference maps showing mean AGB and coefficient of variation (CV) across space
3. Time series graphs showing mean AGB and CV over time

### Pairwise pixel correlations

In [ ]:
# define function to create scatter graph with 1:1 line
def add_scatter(ax, df, x_col, y_col, x_label, y_label):
    ax.scatter(df[x_col], df[y_col], alpha=0.5, s=10, color='darkgreen')
    lims = [min(df[x_col].min(), df[y_col].min()),
            max(df[x_col].max(), df[y_col].max())]
    ax.plot(lims, lims, 'k--', lw=2, alpha=0.7)
    ax.set_xlabel(x_label, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)

# create plotting canvas
fig, axes0 = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Pairwise pixel correspondence', fontsize=14)

# add pairwise pixel plots to canvas
add_scatter(axes0[0], gdf, 'planet_agb', 'chloris_agb', 'Planet AGB (Mg/ha)', 'Chloris AGB (Mg/ha)')
add_scatter(axes0[1], gdf, 'planet_agb', 'kanop_agb', 'Planet AGB (Mg/ha)', 'Kanop AGB (Mg/ha)')
add_scatter(axes0[2], gdf, 'chloris_agb', 'kanop_agb', 'Chloris AGB (Mg/ha)', 'Kanop AGB (Mg/ha)')

# plot
plt.tight_layout()

### Difference maps

In [ ]:
# create plotting canvas
fig, axes1 = plt.subplots(1, 2, figsize=(12, 6))

# map of mean AGB (calculated in SQL query)
gdf.plot(
    column='mean_agb', 
    ax=axes1[0], 
    cmap='viridis', 
    legend=True, 
    legend_kwds={'label': 'Mg/ha', 'shrink': 0.8}
)
axes1[0].set_title('Mean per pixel AGB (Mg/ha)', fontsize=12)

# map of mean per pixel CV
gdf.plot(
    column='cv_agb', 
    ax=axes1[1], 
    cmap='YlOrRd', 
    legend=True,
    legend_kwds={'shrink': 0.8}
)
axes1[1].set_title('Mean per pixel CV', fontsize=12)

# plot
plt.tight_layout()

### Time series graphs

In [ ]:
# create plotting canvas
fig, axes2 = plt.subplots(1, 2, figsize=(12, 6))

# time series of mean AGB (line) ± standard deviation (ribbon)
axes2[0].plot(
    temporal_stats['year'], 
    temporal_stats['mean_agb'], 
    'o-', 
    color='darkgreen'
)
axes2[0].fill_between(
    temporal_stats['year'], 
    temporal_stats['mean_agb'] - temporal_stats['std_mean_agb'],
    temporal_stats['mean_agb'] + temporal_stats['std_mean_agb'],
    alpha=0.3, 
    color='darkgreen'
)
axes2[0].set_ylabel('AGB (Mg/ha ± SD)')
axes2[0].set_xlabel('Year')
axes2[0].set_title('Mean AGB over time (Mg/ha ± SD)')
axes2[0].grid(True, alpha=0.3)

# time series plot of mean CV
axes2[1].plot(
    temporal_stats['year'], 
    temporal_stats['mean_cv'], 
    'o-', 
    color='darkred'
)
axes2[1].set_ylabel('Mean CV')
axes2[1].set_xlabel('Year')
axes2[1].set_title('Mean CV over time')
axes2[1].grid(True, alpha=0.3)

# plot
plt.tight_layout()